# Analysis of all methods results

Here we explore the performance of RL, Planning, and RL+Planning across the different environments.

We will look at their mean performance (i.e. episode returns) across the different environments. Looking at both in-distribution (planning population matches the test population) and out-of-distribution (planning population does not match the test population) settings.

**Note** each experiment run was repeated 5 times (once for each RL policy seed), so we average the results across these 5 runs.

In [ ]:
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from posggym_baselines.config import REPO_DIR

sys.path.insert(0, str(REPO_DIR / "baseline_exps"))
import exp_utils

sns.set_theme()
sns.set_context("paper", font_scale=1.5)
sns.set_palette("colorblind")

SAVE_RESULTS = True

In [ ]:
ALL_ENV_DATA = exp_utils.load_all_env_data()
for k in ALL_ENV_DATA:
    print(k)

NUM_ENVS = len(ALL_ENV_DATA)

# figure parameters
FIGSIZE = (10, 10)
N_COLS = min(3, NUM_ENVS)
N_ROWS = (NUM_ENVS // N_COLS) + int(NUM_ENVS % N_COLS > 0)

## Load Results

In [ ]:
# Add In/Out of Distribution labels
def get_in_out_dist_label(row):
    if row["Algorithm"] in ["POMCP", "INTMCP"]:
        return False
    return bool(row["Planning Population"] == row["Test Population"])

### Planning Results

In [ ]:
planning_results_df = []
for env_id, env_data in ALL_ENV_DATA.items():
    env_planning_results = pd.read_csv(env_data.planning_results_file)
    env_planning_results["full_env_id"] = env_id
    planning_results_df.append(env_planning_results)

planning_results_df = pd.concat(planning_results_df, ignore_index=True)
planning_results_df.rename(
    columns={
        "alg": "Algorithm",
        "planning_pop_id": "Planning Population",
        "test_pop_id": "Test Population",
        "return": "Return",
        "discounted_return": "Discounted Return",
    },
    inplace=True,
)

planning_results_df["In Distribution"] = planning_results_df.apply(
    get_in_out_dist_label, axis=1
)

planning_results_df.sort_values(
    by=[
        "Algorithm", 
        "full_env_id", 
        "Planning Population", 
        "Test Population", 
        "search_time_limit"
    ], 
    inplace=True
)

max_search_time = planning_results_df["search_time_limit"].max()

planning_results_df = planning_results_df[planning_results_df["search_time_limit"].isin([0.05, 0.1, 0.5, 1, 5, 10, 20])]
print(planning_results_df["search_time_limit"].unique())

### RL Results

In [ ]:
br_results_df = []
for full_env_id, env_data in ALL_ENV_DATA.items():
    env_br_results_df = pd.read_csv(env_data.rl_br_results_file)
    env_br_results_df["full_env_id"] = full_env_id
    br_results_df.append(env_br_results_df)

br_results_df = pd.concat(br_results_df, ignore_index=True)
br_results_df.rename(
    columns={
        "train_pop": "Planning Population",
        "test_pop": "Test Population",
        "return": "Return",
        "discounted_return": "Discounted Return",
        "train_seed": "rl_policy_seed"
    },
    inplace=True,
)

br_results_df["Algorithm"] = "RL-BR"
br_results_df["In Distribution"] = br_results_df.apply(
    get_in_out_dist_label, axis=1
)


br_results_df.sort_values(
    by=[
        "full_env_id", 
        "Planning Population", 
        "Test Population"
    ], 
    inplace=True
)

### Combined (RL+Planning) Results

In [ ]:
combined_results_df = []
for env_id, env_data in ALL_ENV_DATA.items():
    env_planning_results = pd.read_csv(env_data.combined_results_file)
    env_planning_results["full_env_id"] = env_id
    combined_results_df.append(env_planning_results)

combined_results_df = pd.concat(combined_results_df, ignore_index=True)
combined_results_df.rename(
    columns={
        "alg": "Algorithm",
        "planning_pop_id": "Planning Population",
        "test_pop_id": "Test Population",
        "return": "Return",
        "discounted_return": "Discounted Return",
    },
    inplace=True,
)


combined_results_df["In Distribution"] = combined_results_df.apply(
    get_in_out_dist_label, axis=1
)

combined_results_df.sort_values(
    by=[
        "Algorithm", 
        "full_env_id", 
        "Planning Population", 
        "Test Population", 
        "search_time_limit"
    ], 
    inplace=True
)

assert (combined_results_df["rl_policy_pop_id"] == combined_results_df["Planning Population"]).all()
combined_results_df.drop(columns=["rl_policy_pop_id"], inplace=True)

# combined_results_df = combined_results_df[combined_results_df["search_time_limit"].isin([0.05, 0.1, 0.5, 1, 5, 10, 20])]
print(combined_results_df["search_time_limit"].unique())

### All results

In [ ]:
# Need to add search_time_limit to br_results_df
# We duplicate the DF and set the search_time_limit to the max and min values
# This will produce a horizontal line in the plots :)
min_search_time = min(
    planning_results_df["search_time_limit"].min(),
    combined_results_df["search_time_limit"].min(),
)
max_search_time = max(
    planning_results_df["search_time_limit"].max(),
    combined_results_df["search_time_limit"].max(),
)
br_results_df["search_time_limit"] = max_search_time
min_br_results_df = br_results_df.copy(deep=True)
min_br_results_df["search_time_limit"] = min_search_time

# combine planning, combined, and rl results together
all_results_df = pd.concat(
    [
        planning_results_df, 
        combined_results_df, 
        br_results_df,
        min_br_results_df
    ],
    ignore_index=True
)

all_results_df.sort_values(
    by=[
        "Algorithm", 
        "full_env_id", 
        "Planning Population", 
        "Test Population", 
        "search_time_limit"
    ], 
    inplace=True
)

all_results_df["rl_policy_seed"] = all_results_df["rl_policy_seed"].astype("category")
all_results_df["num"] = all_results_df["num"].astype("category")

### Generalization results

In [ ]:
# remove unused columns
gg_df = all_results_df[
    [
        "Algorithm",
        "full_env_id",
        "Planning Population",
        "Test Population",
        "search_time_limit",
        "Return",
        "In Distribution",
        # "rl_policy_seed",
    ]
]
gg_df = gg_df[
    gg_df["Algorithm"].isin(["COMBINED", "RL-BR", "IPOMCP", "POTMMCP"])
]

# average over episodes
gg_gb = gg_df.groupby(
    [
        "Algorithm",
        "full_env_id",
        "Planning Population",
        "Test Population",
        "search_time_limit",
        "In Distribution",
    ]
).agg({"Return": ["mean", "count", "std", "var"]}).reset_index()
gg_gb.columns = ["_".join(x) if x[1] != '' else x[0] for x in gg_gb.columns.ravel()]
# gg_gb

gg_df = gg_gb[
    gg_gb["In Distribution"] == True
].merge(
    gg_gb[gg_gb["In Distribution"] == False],
    on=[
        "Algorithm",
        "full_env_id",
        "Planning Population",
        "search_time_limit",
        # "rl_policy_seed",
    ],
    suffixes=("", "_test"),
)
gg_df["Generalization Gap"] = gg_df["Return_mean"] - gg_df["Return_mean_test"]
gg_df["Generalization Gap CI"] = 1.96 * np.sqrt(
    gg_df["Return_var"] / gg_df["Return_count"]
    + gg_df["Return_var_test"] / gg_df["Return_count_test"]
)
print(gg_df["Algorithm"].unique())
gg_df


## Formatting

In [ ]:
alg_order = [
    "INTMCP",
    "IPOMCP",
    "POMCP",
    "POTMMCP",
    "RL-BR",
    "COMBINED",
]
# ref: https://seaborn.pydata.org/tutorial/color_palettes.html
alg_color_palette = sns.color_palette("tab10")
alg_palette={
    # planning methods
    "INTMCP": alg_color_palette[0],
    "IPOMCP": alg_color_palette[1],
    "POMCP": alg_color_palette[2],
    "POTMMCP": alg_color_palette[3],
    # rl
    "RL-BR": alg_color_palette[4],
    # combined
    "COMBINED": alg_color_palette[6],
}
alg_dashes = {
    # planning methods
    "INTMCP": (2, 2),
    "IPOMCP": (2, 2),
    "POMCP": (2, 2),
    "POTMMCP": (2, 2),
    # rl
    "RL-BR": "",
    # combined
    "COMBINED": (1, 1),
}
alg_dashes = {
    # planning methods
    "INTMCP": "",
    "IPOMCP": "",
    "POMCP": "",
    "POTMMCP": "",
    # rl
    "RL-BR": (2, 2),
    # combined
    "COMBINED": (4, 2),
}

## All Results

Here we show In and Out-of distribution performance for each method in each environment in a single plot.

In [ ]:
g = sns.relplot(
    data=all_results_df,
    x="search_time_limit",
    y="Return",
    hue="Algorithm",
    style="Algorithm",
    row="full_env_id",
    col="In Distribution",
    col_order=[True, False],
    hue_order=alg_order,
    legend="full",
    # palette="tab10",
    # palette="colorblind",
    palette=alg_palette,
    markers=False,
    dashes=alg_dashes,
    kind="line",
    height=3,
    aspect=1.5,
    facet_kws={
        "sharey": 'row', 
        "sharex": True
    },
)

for (row_key, col_key), ax in g.axes_dict.items():
    col_key = "Out of Dist." if bool(col_key) is False else "In Dist."
    ax.set_title(f"{row_key} | {col_key}")
    ax.set_xlabel("Search Time Limit (s)")
    ax.set_ylabel("Mean Return")

del g

## In-Distribution Performance

Here we look specifically at the in-distribution performance of all methods (planning population and test populations are the same). We look only at the performance of each algorithm given the maximum search time.

Dimensions:

- `y-axis`: mean episode return
- `x-axis`: search time
- `z-axis/hue`: algorithm

In [ ]:
g = sns.relplot(
    data=all_results_df[all_results_df["In Distribution"] == True],
    x="search_time_limit",
    y="Return",
    hue="Algorithm",
    style="Algorithm",
    col="full_env_id",
    col_wrap=3,
    hue_order=[a for a in alg_order if a not in ["INTMCP", "POMCP"]],
    legend="full",
    palette=alg_palette,
    markers=False,
    dashes=alg_dashes,
    kind="line",
    height=4,
    aspect=1,
    facet_kws={
        "sharey": False, 
        "sharex": False
    },
)

for i, (col_key, ax) in enumerate(g.axes_dict.items()):
    ax.set_title(col_key)
    if i % 3 == 0:
        ax.set_ylabel("Mean Return")
    if i >= 3:
        ax.set_xlabel("Search Time (s)")

if SAVE_RESULTS:
    print("saving figure")
    g.savefig(
        exp_utils.ENV_DATA_DIR / "figures" / "all_methods_returns_ID.pdf", 
        bbox_inches="tight"
    )
del g

In [ ]:
g = sns.relplot(
    data=all_results_df[all_results_df["In Distribution"] == True],
    x="search_time_limit",
    y="Return",
    hue="Algorithm",
    style="Algorithm",
    row="full_env_id",
    col="Planning Population",
    hue_order=[a for a in alg_order if a not in ["INTMCP", "POMCP"]],
    legend="full",
    palette=alg_palette,
    markers=False,
    dashes=alg_dashes,
    kind="line",
    height=3,
    aspect=1.5,
    facet_kws={
        "sharey": "row", 
        "sharex": True
    },
)

for col_key, ax in g.axes_dict.items():
    ax.set_title(col_key)
    ax.set_xlabel("Search Time Limit (s)")
    ax.set_ylabel("Mean Return")

del g

## Out-of-Distribution Performance

In [ ]:
g = sns.relplot(
    data=all_results_df[all_results_df["In Distribution"] == False],
    x="search_time_limit",
    y="Return",
    hue="Algorithm",
    style="Algorithm",
    col="full_env_id",
    col_wrap=3,
    hue_order=alg_order,
    legend="full",
    palette=alg_palette,
    markers=False,
    dashes=alg_dashes,
    kind="line",
    height=4,
    aspect=1,
    facet_kws={
        "sharey": False, 
        "sharex": False
    },
)

for i, (col_key, ax) in enumerate(g.axes_dict.items()):
    ax.set_title(col_key)
    if i % 3 == 0:
        ax.set_ylabel("Mean Return")
    if i >= 3:
        ax.set_xlabel("Search Time (s)")

if SAVE_RESULTS:
    print("saving figure")
    g.savefig(
        exp_utils.ENV_DATA_DIR / "figures" / "all_methods_returns_OOD.pdf", 
        bbox_inches="tight"
    )
del g

In [ ]:
g = sns.relplot(
    data=all_results_df[all_results_df["In Distribution"] == False],
    x="search_time_limit",
    y="Return",
    hue="Algorithm",
    style="Algorithm",
    row="full_env_id",
    col="Planning Population",
    hue_order=alg_order,
    legend="full",
    palette=alg_palette,
    markers=False,
    dashes=alg_dashes,
    kind="line",
    height=3,
    aspect=1.5,
    facet_kws={
        "sharey": "row", 
        "sharex": True
    },
)

for col_key, ax in g.axes_dict.items():
    ax.set_title(col_key)
    ax.set_xlabel("Search Time Limit (s)")
    ax.set_ylabel("Mean Return")

del g

## In vs Out of Distribution Performance

Here we look at the gap between in-distribution and out-of-distribution performance of each algorithm.

In the first plot we show the mean return of each algorithm for in and out of distribution settings, averaged across all environments.

- `x-axis`: Search Time
- `y-axis`: Generalization Gap (In - Out)
- `hue/z-axis`: In Distribution
- `col`: Algorithm

In [ ]:
ID_VS_OOD_ALGS = ["COMBINED", "RL-BR", "IPOMCP", "POTMMCP"]
g = sns.relplot(
    data=all_results_df[
        all_results_df["Algorithm"].isin(ID_VS_OOD_ALGS)
    ],
    x="search_time_limit",
    y="Return",
    hue="In Distribution",
    col="Algorithm",
    # col_wrap=3,
    legend="full",
    kind="line",
    height=4,
    aspect=1,
    facet_kws={
        "sharey": "row", 
        "sharex": False
    },
)

for i, (col_key, ax) in enumerate(g.axes_dict.items()):
    ax.set_title(f"{col_key}")
    if i % len(ID_VS_OOD_ALGS) == 0:
        ax.set_ylabel("Mean Return")
    if i >= len(g.axes_dict) - len(ID_VS_OOD_ALGS):
        ax.set_xlabel("Search Time (s)")

if SAVE_RESULTS:
    print("saving figure")
    g.savefig(
        exp_utils.ENV_DATA_DIR / "figures" / "all_methods_returns_ID_vs_OOD.pdf", 
        bbox_inches="tight"
    )
del g

Here we do the same plot but this time for each environment separately.

- `x-axis`: Search Time
- `y-axis`: Generalization Gap (In - Out)
- `hue/z-axis`: In Distribution
- `col`: Algorithm
- `row`: Environment

In [ ]:
ID_VS_OOD_ALGS = ["COMBINED", "RL-BR", "IPOMCP", "POTMMCP"]
g = sns.relplot(
    data=all_results_df[
        all_results_df["Algorithm"].isin(ID_VS_OOD_ALGS)
    ],
    x="search_time_limit",
    y="Return",
    hue="In Distribution",
    col="Algorithm",
    row="full_env_id",
    legend="full",
    kind="line",
    height=3,
    aspect=1.5,
    facet_kws={
        "sharey": "row", 
        "sharex": False
    },
)

for i, ((row_key, col_key), ax) in enumerate(g.axes_dict.items()):
    ax.set_title(f"{row_key} | {col_key}")
    if i % len(ID_VS_OOD_ALGS) == 0:
        ax.set_ylabel("Mean Return")
    if i >= len(g.axes_dict) - len(ID_VS_OOD_ALGS):
        ax.set_xlabel("Search Time (s)")

if SAVE_RESULTS:
    print("saving figure")
    g.savefig(
        exp_utils.ENV_DATA_DIR / "figures" / "all_methods_returns_ID_vs_OOD_allenvs.pdf", 
        bbox_inches="tight"
    )
del g

- `x-axis`: Search Time
- `y-axis`: Generalization Gap (In - Out)
- `hue/z-axis`: Algorithm
- `col/figures`: Environment

In [ ]:
# remove unused columns
gg_df = all_results_df[
    [
        "Algorithm",
        "full_env_id",
        "Planning Population",
        "Test Population",
        "search_time_limit",
        "Return",
        "In Distribution",
        # "rl_policy_seed",
    ]
]
gg_df = gg_df[
    gg_df["Algorithm"].isin(["COMBINED", "RL-BR", "IPOMCP", "POTMMCP"])
]

# average over planning population and episodes
gg_gb = gg_df.groupby(
    [
        "Algorithm",
        "full_env_id",
        # "Planning Population",
        # "Test Population",
        "search_time_limit",
        "In Distribution",
    ]
).agg({"Return": ["mean", "count", "std", "var"]}).reset_index()
gg_gb.columns = ["_".join(x) if x[1] != '' else x[0] for x in gg_gb.columns.ravel()]
# gg_gb

gg_df = gg_gb[
    gg_gb["In Distribution"] == True
].merge(
    gg_gb[gg_gb["In Distribution"] == False],
    on=[
        "Algorithm",
        "full_env_id",
        # "Planning Population",
        "search_time_limit",
        # "rl_policy_seed",
    ],
    suffixes=("", "_test"),
)
gg_df["Generalization Gap"] = gg_df["Return_mean"] - gg_df["Return_mean_test"]
gg_df["Generalization Gap CI"] = 1.96 * np.sqrt(
    gg_df["Return_var"] / gg_df["Return_count"]
    + gg_df["Return_var_test"] / gg_df["Return_count_test"]
)
print(gg_df["Algorithm"].unique())
gg_df

In [ ]:
g = sns.relplot(
    data=gg_df,
    x="search_time_limit",
    y="Generalization Gap",
    hue="Algorithm",
    style="Algorithm",
    col="full_env_id",
    errorbar=None,
    col_wrap=3,
    hue_order=alg_order,
    legend="full",
    palette=alg_palette,
    markers=False,
    dashes=alg_dashes,
    kind="line",
    height=4,
    aspect=1,
    facet_kws={
        "sharey": False, 
        "sharex": False
    },
)
g.refline(x=None, y=0.0, linestyle="-", linewidth=0.5)

for i, (col_key, ax) in enumerate(g.axes_dict.items()):
    ax.set_title(col_key)

    # add custome error bars
    gg_df_env = gg_df[gg_df["full_env_id"] == col_key]
    for alg in gg_df_env["Algorithm"].unique():
        gg_df_alg = gg_df_env[gg_df_env["Algorithm"] == alg]

        gg_df_alg = gg_df_alg.sort_values(by=["search_time_limit"])
        ax.fill_between(
            x=gg_df_alg["search_time_limit"],
            y1=gg_df_alg["Generalization Gap"] - gg_df_alg["Generalization Gap CI"],
            y2=gg_df_alg["Generalization Gap"] + gg_df_alg["Generalization Gap CI"],
            color=alg_palette[alg],
            alpha=0.2,
        )

    if i % 3 == 0:
        ax.set_ylabel("Generalization Gap (ID - OOD)")
    if i >= 3:
        ax.set_xlabel("Search Time (s)")

# if SAVE_RESULTS:
#     g.savefig(
#         exp_utils.ENV_DATA_DIR / "figures" / "all_methods_returns_OOD.pdf", 
#         bbox_inches="tight"
#     )

del g

## Search Statistics

Here we look at various statistics of the search process for each algorithm.

Each figure is a different statistic, each line is a different environment since we expect some differences between environments based on things like average steps to terminal state. In and out of distribution results are grouped together since we expect and see no different in search statistics between in vs out of distribution.

- `x-axis`: Search time
- `y-axis`: Search Statistic Values
- `col/figures`: Environment

In [ ]:
for stat_key in [
    "search_time",
    "update_time",
    "reinvigoration_time",
    "evaluation_time",
    "policy_calls",
    "inference_time",
    "search_depth",
    "num_sims",
    "mem_usage",
    "min_value",
    "max_value",
]:
    print(stat_key)
    stat_df = all_results_df[all_results_df["Algorithm"] != "RL-BR"]
    stat_min = stat_df[stat_key].min()
    stat_max = stat_df[stat_key].max()
    use_log_scale = (stat_max / max(1, stat_min)) > 100 and stat_key != "min_value"

    g = sns.relplot(
        data=all_results_df[all_results_df["Algorithm"] != "RL-BR"],
        x="search_time_limit",
        y=stat_key,
        hue="Algorithm",
        # col_wrap=N_COLS,
        row="Planning Population",
        col="full_env_id",
        kind="line",
        facet_kws={
            "sharey": True, 
            "sharex": True,
        },
    )

    for (row_key, col_key), ax in g.axes_dict.items():
        ax.set_title(f"{row_key} | {col_key}")
        ax.set_xlabel("Search Time Limit (s)")
        if use_log_scale:
            ax.set_yscale("log")

    del g